# Lab2: Data analysis

Throughout all the following labs, we will work on the following problem: 

**Is it possible to predict which Pokemon is legendary given their attributes ?**

We will begin by analyzing it using `pandas` and plot the data using `matplotlib` and `seaborn`.

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

/tmp/ipykernel_31002/3918063340.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Load dataset
The dataset is available in its `csv` format.

In [2]:
df = pd.read_csv("../pokemon.csv")
df.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


## Identify rows and columns
1. Find the number of rows and the number of columns

In [5]:
df.shape

(801, 41)

2. List every column name and give a brief description of the data, helping yourself with the [kaggle description](https://www.kaggle.com/datasets/rounakbanik/pokemon) of the dataset.

In [6]:
df.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

3. Specify for each column the data type and create a list `quantitative_vars` that will contain the names of the quantitative variables and `qualitative_vars` that will contain the names of the qualitative variables.

In [11]:
qualitative_vars = ["abilities", "classfication", "pokedex_number", "japanese_name", "name", "type1", "type2","generation"]
quantitative_vars = [col for col in df.columns if col not in qualitative_vars + ["is_legendary"]]

In [12]:
quantitative_vars

['against_bug',
 'against_dark',
 'against_dragon',
 'against_electric',
 'against_fairy',
 'against_fight',
 'against_fire',
 'against_flying',
 'against_ghost',
 'against_grass',
 'against_ground',
 'against_ice',
 'against_normal',
 'against_poison',
 'against_psychic',
 'against_rock',
 'against_steel',
 'against_water',
 'attack',
 'base_egg_steps',
 'base_happiness',
 'base_total',
 'capture_rate',
 'defense',
 'experience_growth',
 'height_m',
 'hp',
 'percentage_male',
 'sp_attack',
 'sp_defense',
 'speed',
 'weight_kg']

**3. Bonus question** Create new variables that seem relevant for your analysis.

4. What column are we trying to predict ? Store it into a list `target`.

In [13]:
target = df["is_legendary"]

5. Infer what columns will be used for prediction and what column you can omit immediately. Create the corresponding list `features` and update `quantitative_vars` and `qualitative_vars`.

In [16]:
qualitative_vars = [var for var in qualitative_vars if var not in ["name", "japanese_name", "pokedex_number"]]

features = df[qualitative_vars+quantitative_vars]

In [17]:
features.head()

,abilities,classfication,type1,type2,generation,against_bug,against_dark,against_dragon,against_electric,against_fairy,...,capture_rate,defense,experience_growth,height_m,hp,percentage_male,sp_attack,sp_defense,speed,weight_kg
0,"['Overgrow', 'Chlorophyll']",Seed Pokémon,grass,poison,1,1.0,1.0,1.0,0.5,0.5,...,45,49,1059860,0.7,45,88.1,65,65,45,6.9
1,"['Overgrow', 'Chlorophyll']",Seed Pokémon,grass,poison,1,1.0,1.0,1.0,0.5,0.5,...,45,63,1059860,1.0,60,88.1,80,80,60,13.0
2,"['Overgrow', 'Chlorophyll']",Seed Pokémon,grass,poison,1,1.0,1.0,1.0,0.5,0.5,...,45,123,1059860,2.0,80,88.1,122,120,80,100.0
3,"['Blaze', 'Solar Power']",Lizard Pokémon,fire,NaN,1,0.5,1.0,1.0,1.0,0.5,...,45,43,1059860,0.6,39,88.1,60,50,65,8.5
4,"['Blaze', 'Solar Power']",Flame Pokémon,fire,NaN,1,0.5,1.0,1.0,1.0,0.5,...,45,58,1059860,1.1,58,88.1,80,65,80,19.0


6. Give the repartition of the target to predict using the `value_counts` method. How many pokemons are legendary ?


In [18]:
target.value_counts()

is_legendary
0    731
1     70
Name: count, dtype: int64

## Univariate analysis

1. For each **quantitative variables** (that you should have stored in the variable `quantitative_vars`), give the main statistical estimators: mean, median, standard error.



2. For each **quantitative variables** (that you should have stored in the variable `quantitative_vars`), plot the variable repartitions using `sns.histplot` and `sns.violinplot`. Deduce for each variable a possible distribution.

3. For each **qualitative variables** (that you should have stored in the variable `qualititative_vars`), provide the different values that can be taken by the variable and count the number of individuals per category.

## Multivariate analysis

### Correlation of variables

1] Using the `.corr()` method, find and analyze correlations between features. What can you deduce regarding features independence ?

2] Plot the correlogram and perform a quick analysis of relationship between variables using `sns.pairplot`. Understand graph per graph what is represented and deduce from it the relationship between features and what feature seems to be important for prediction.

**3] Bonus question**: use the `sns.pairplot` function to check the correlation between qualitative variables as well.

### Impact of variables/features on target

The goal of our study is to **predict the target is_legendary**, so we need to analyze the relationship between this target and the features.

1] Find the statistical repartition of **quantitative variables** (that you should have stored in the variable `quantitative_vars`) for legendary and non-legendary pokemons and plot it using `sns.boxplot()`. 

2] Find the statistical repartition of is_legendary for each **categorical variable** (that you should have stored in the variable `qualitative_vars`) and plot it using the `sns.barplot()` function.

## Dealing with missing values

1] Have you noticed anything in particular with missing values ?

2] Find rows with missing values using the `df.isna()`` method and perform a statistical analysis on the number of missing values.

3] Suggest possible ways to deal with missing values and apply it to the data.


## Conclusions and next steps

1. What seems to be the most impacting features in the dataset to predict what Pokemon is legendary ?


2. Do you think some metrics could be removed from the dataset without information loss ?


4. Store this new dataset into a csv file `pokemon_preprocessed` (using the `to_csv` method) and save it for the next lab.